# Perceptron Learning Algorithm (Binary Classifier)

### Modification History
\>\>\> ZJ Zhang (Jan 23th, 2017) 

## Overview
Given $N$ input vectors with $D$ dimensions as training data, with each noted as $\boldsymbol{x_{i}}$ and known labels of $l_{i}$ (binary values of $-1$ or $1$), I will build a perceptron that the output of each $\boldsymbol{x_{i}}$ is exactly its corresponding label of $l_{i}$. Each vector $\boldsymbol{x_{i}}$ could be expressed as

$$
\boldsymbol{x_{i}} = (x_{i,1}, x_{i,2}, ..., x_{i,D})
$$
with each element termed $x_{i,j}$ ($j=1,2,...,D$)

In [ ]:
# import packages
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

## Training Data
Training data are created by prior knowledge. I assume there are $N$ vectors (or points) serving as training data, randomly distributed within a $D$-dimensional sphere with a radius of $R$. Then I assign them labels based on a linear boundary, described by 
$$
p_{0} + p_{1}x_{1} + p_{2}x_{2} + ... + p_{D}x_{D} = 0 \quad \Rightarrow \quad \boldsymbol{m_{j}}\boldsymbol{x_{j}}=0
$$
where,
$$
\boldsymbol{p} = (p_{0}, p_{1}, p_{2}, ..., p_{D}) \\
\boldsymbol{x} = (1, x_{1}, x_{2}, ..., x_{D})
$$
**If an vector point is above the boundary, then I assign a label of $1$, otherwise (i.e., below or right on the boundary), I assign a label of $-1$.**

Here I assume all data are uniformly distributed in the hypersphere. In addition, the boundary parameters $\boldsymbol{p}$ are generated randomly within $[p_{\rm min}, p_{\rm max}]$ with a dimension of $D+1$. The following parameters are defiend by the user: $N$, $D$, $R$, $p_{\rm bound} = [p_{\rm min}, p_{\rm max}]$. 

In [ ]:
# metric of the training data
Nd = 1000 # number of vectors/points
Dd = 2   # dimension
Rd = 100  # radius of the hypersphere where all data are distributed
p_bound = [-10, 10]  # extent of the boundary parameters

# generate the prior boundary parameter p
p_param = np.random.uniform(low=p_bound[0], high=p_bound[-1], size=Dd+1)

In [ ]:
# training data generation
Train_Data = np.random.uniform(low=-Rd, high=Rd, size=(Nd,Dd+1))
Train_Data[:,0] = 1.0  # append a 1.0 to the beginning of each vectors' coordinates
# assign labels based on points' position compared to the prior boundary
Train_boolLabel = np.sum(Train_Data*p_param, axis=1) > 0  # points located above the boundary have "True" with remaining have "False"
Train_Label = 2 * Train_boolLabel - 1  # convert "True" and "False" to "1" and "-1", respectively

In [ ]:
# plot the training data and the prior boundary - for 2D only
if Dd==2:
    # marker geometry
    marker = 'o'
    markersize = 10
    markeredgewidth = 1
    # colors
    pos_color = 'goldenrod'
    neg_color = 'lightgrey'
    edgecolor = 'k'
    # lines
    marker_linestyle = 'None'
    bound_linestyle = '-'
    bound_linewdith = 4
    prbound_color = 'blue'
    # plot points with different labels
    pos_id = np.where(Train_Label==1)[0]
    plt.plot(Train_Data[pos_id,1], Train_Data[pos_id,2], marker=marker, 
             markersize=markersize, markerfacecolor=pos_color, markeredgecolor=edgecolor, 
             markeredgewidth=markeredgewidth, linestyle=marker_linestyle, label='label=1')
    neg_id = np.where(Train_Label==-1)[0]
    plt.plot(Train_Data[neg_id,1], Train_Data[neg_id,2], marker=marker, 
             markersize=markersize, markerfacecolor=neg_color, markeredgecolor=edgecolor, 
             markeredgewidth=markeredgewidth, linestyle=marker_linestyle, label=u'label=\u22121')
    # plot the prior boundary
    x_bound = np.linspace(-Rd, Rd,2)
    y_prbound = -1.0 * (p_param[0] + p_param[1]*x_bound) / p_param[2]
    plt.plot(x_bound, y_prbound, color=prbound_color, linestyle=bound_linestyle, 
             linewidth=bound_linewdith, label='prior boundary')
    # limits and label
    plt.xlim([-Rd, Rd])
    plt.ylim([-Rd, Rd])
    # legend
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, numpoints=1, mode="expand", borderaxespad=0.)

## Algorithm
The perceptron learning is actually to find the (linear) boundary that separate the vectors/points with two labels, given the training data. The whole algorithm include the following steps:

1) Initialize a boundary parameter $\boldsymbol{w}$, weights.

2) For misclassified examples (e.g., $\boldsymbol{x_{j}}$ whose $l_{j} \neq y_{j}$), compute the classifier output ($y_{j}$) and update the weights based on $\boldsymbol{w} += c (l_{j} - y_{j}) \boldsymbol{x_{j}}$, where $c$ is the learning rate.

3) Iterate among all of the training data until the output classification of all data match their own labels.

In [ ]:
# Build up a perceptron - Proof of Concept
# learning rate
learn_rate = np.pi
# initialize the boundary parameter
w_bound = [-100, 100]
w_param = np.random.uniform(low=w_bound[0], high=w_bound[-1], size=Dd+1)
# success number
success_num = 0
# index of scanning the training data
index = 0
while success_num<Nd:
    # decide if the current data has correct model-predicted label
    Model_boolLabel_cur = np.sum(w_param * Train_Data[index]) > 0
    Model_Label_cur = 2 * Model_boolLabel_cur - 1
    if Model_Label_cur==Train_Label[index]:
        success_num += 1
    else:
        sucess_num = 0
        w_param += learn_rate * (Train_Label[index] - Model_Label_cur) * Train_Data[index]
    index = (index+1)%Nd

In [ ]:
# compare the model-predicted boundary with the prior boundary - for 2D only
if Dd==2:
    # marker geometry
    marker = 'o'
    markersize = 10
    markeredgewidth = 1
    # colors
    pos_color = 'goldenrod'
    neg_color = 'lightgrey'
    edgecolor = 'k'
    # lines
    marker_linestyle = 'None'
    bound_linestyle = '-'
    bound_linewdith = 4
    prbound_color = 'blue'
    compbound_color = 'red'
    # plot points with different labels
    pos_id = np.where(Train_Label==1)[0]
    plt.plot(Train_Data[pos_id,1], Train_Data[pos_id,2], marker=marker, 
             markersize=markersize, markerfacecolor=pos_color, markeredgecolor=edgecolor, 
             markeredgewidth=markeredgewidth, linestyle=marker_linestyle, label='label=1')
    neg_id = np.where(Train_Label==-1)[0]
    plt.plot(Train_Data[neg_id,1], Train_Data[neg_id,2], marker=marker, 
             markersize=markersize, markerfacecolor=neg_color, markeredgecolor=edgecolor, 
             markeredgewidth=markeredgewidth, linestyle=marker_linestyle, label=u'label=\u22121')
    # plot the prior boundary
    x_bound = np.linspace(-Rd, Rd,2)
    y_prbound = -1.0 * (p_param[0] + p_param[1]*x_bound) / p_param[2]
    plt.plot(x_bound, y_prbound, color=prbound_color, linestyle=bound_linestyle, 
             linewidth=bound_linewdith, label='prior boundary')
    # plot the model-predicted boundary
    y_compbound = -1.0 * (w_param[0] + w_param[1]*x_bound) / w_param[2]
    plt.plot(x_bound, y_compbound, color=compbound_color, linestyle=bound_linestyle, 
             linewidth=bound_linewdith, label='model-predicted boundary')
    # limits and label
    plt.xlim([-Rd, Rd])
    plt.ylim([-Rd, Rd])
    # legend
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, numpoints=1, mode="expand", borderaxespad=0.)

## Investigations
Here I investigate the following few points:

1) The maximum interation number $N_{\rm iter,max}$ as a function of learning rate $R_{\rm learn}$

In [ ]:
# Here I build up the perceptron again, by including more components
# initialize the boundary parameter
w_bound = [-100, 100]
w_param = np.random.uniform(low=w_bound[0], high=w_bound[-1], size=Dd+1)
# list of learning rates
Rlearn = np.array([1.0e-4, 3.0e-4, 1.0e-3, 3.0e-3, 1.0e-2, 3.0e-2, 1.0e-1, 3.0e-1, 
                        1.0, 3.0, 10., 30., 1.0e2, 3.0e2, 1.0e3, 3.0e3, 1.0e4])
# create an array to save the maximum iterations and the final weights
MaxNiter = np.ones_like(Rlearn) * np.nan
Weights = np.ones((len(Rlearn), Dd+1)) * np.nan
for index_learn in range(0,len(Rlearn)):
    # iteration step
    N_iter = 0
    # success number
    success_num = 0
    # index of scanning the training data
    index = 0
    while success_num<Nd:
        # decide if the current data has correct model-predicted label
        Model_boolLabel_cur = np.sum(w_param * Train_Data[index]) > 0
        Model_Label_cur = 2 * Model_boolLabel_cur - 1
        if Model_Label_cur==Train_Label[index]:
            success_num += 1
        else:
            sucess_num = 0
            w_param += Rlearn[index_learn] * (Train_Label[index] - Model_Label_cur) * Train_Data[index]
        index = (index+1)%Nd
        # update iteration number
        N_iter += 1
    # save the maximum iterations and final weights
    MaxNiter[index_learn] = N_iter
    Weights[index_learn] = w_param

In [ ]:
plt.plot(np.log10(Rlearn), MaxNiter)
plt.xlabel(r'log$_{10}$(Learning Rate)')
plt.ylabel('The Maximum Iterations')